In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from fipy import FaceVariable, CellVariable, Gmsh2DIn3DSpace, Viewer, TransientTerm, DiffusionTerm, DefaultSolver, GaussianNoiseVariable
from fipy.variables.variable import Variable
from fipy.tools import numerix
#from joblib import Parallel, delayed
import multiprocessing

In [3]:
# define mesh
mesh = Gmsh2DIn3DSpace('''
    radius = 1.0;
    cellSize = 0.1;
    // create inner 1/8 shell
    Point(1) = {0, 0, 0, cellSize};
    Point(2) = {-radius, 0, 0, cellSize};
    Point(3) = {0, radius, 0, cellSize};
    Point(4) = {0, 0, radius, cellSize};
    Circle(1) = {2, 1, 3};
    Circle(2) = {4, 1, 2};
    Circle(3) = {4, 1, 3};
    Line Loop(1) = {1, -3, 2} ;
    Ruled Surface(1) = {1};
    // create remaining 7/8 inner shells
    t1[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{1};}};
    t2[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{1};}};
    t3[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{1};}};
    t4[] = Rotate {{0,1,0},{0,0,0},-Pi/2} {Duplicata{Surface{1};}};
    t5[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{t4[0]};}};
    t6[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{t4[0]};}};
    t7[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{t4[0]};}};
    // create entire inner and outer shell
    Surface Loop(100)={1,t1[0],t2[0],t3[0],t7[0],t4[0],t5[0],t6[0]};
''', order=2).extrude(extrudeFunc=lambda r: 1.001 * r) # doctest: +GMSH


In [4]:
mmag = FaceVariable(name=r"$mmag$", mesh=mesh) # doctest: +GMSH

In [5]:
phi = CellVariable(name=r"$\phi$", mesh=mesh) # doctest: +GMSH

phi.setValue(GaussianNoiseVariable(mesh=mesh,
                                   mean=0.0,
                                   variance=1.0)) # doctest: +GMSH

In [6]:
phi

$\phi$

In [7]:
gridCoor = mesh.faceCenters

In [8]:
type(gridCoor)

fipy.variables.faceVariable.FaceVariable

In [9]:
len(gridCoor[0])

12705

In [10]:
numerix.shape(gridCoor)

(3, 12705)

In [11]:
if __name__ == "__main__":
    try:
        viewer = MayaviClient(vars=phi,
                              datamin=0., datamax=1.,
                              daemon_file="examples/cahnHilliard/sphereDaemon.py")
    except:
        viewer = Viewer(vars=phi,
                        datamin=0., datamax=1.,
                        xmin=-2.5, zmax=2.5)